# Linear Discriminant Analysis

Import neccessary libraries and create the dataset

In [3]:
import numpy as np
xp = np.array([[4,2,2,3,4,6,3,8], [1,4,3,6,4,2,2,3], [0,1,1,0,-1,0,1,0]])
xn = np.array([[9,6,9,8,10], [10,8,5,7,8], [1,0,0,1,-1]])
print(xp.T)
print(xn.T)

[[ 4  1  0]
 [ 2  4  1]
 [ 2  3  1]
 [ 3  6  0]
 [ 4  4 -1]
 [ 6  2  0]
 [ 3  2  1]
 [ 8  3  0]]
[[ 9 10  1]
 [ 6  8  0]
 [ 9  5  0]
 [ 8  7  1]
 [10  8 -1]]


1. Function to compute the mean

In [4]:
def computeMean(x):
    return x.mean(0)

2. Use function to compute the mean of our dataset

In [5]:
meanXp = computeMean(xp.T)
meanXn = computeMean(xn.T)
print(meanXp)
print(meanXn)

[4.    3.125 0.25 ]
[8.4 7.6 0.2]


3. Function to compute the covariance matrix of our dataset

In [6]:
def covar(x, mean):
    data = x.T-mean
    dataTransposed = data.T
    dataTransposedTwo = dataTransposed
    covar = np.zeros((np.size(data,1), np.size(data,1)))
    for i, r in enumerate(dataTransposed):
        for j, c in enumerate(dataTransposedTwo):
            covar[i][j] = ((np.dot(c,r)) / (r.size-1))
    return covar

4. Compute the covariance matrix of our dataset

In [7]:
covarXp = covar(xp, meanXp)
covarXn = covar(xn, meanXn)
print(covarXp)
print(covarXn)

[[ 4.28571429 -0.85714286 -0.71428571]
 [-0.85714286  2.41071429 -0.17857143]
 [-0.71428571 -0.17857143  0.5       ]]
[[ 2.3  -0.05 -0.35]
 [-0.05  3.3   0.35]
 [-0.35  0.35  0.7 ]]


5. Function to compute between class scattering matrix Sb

In [8]:
def betweenScatter (xp, xn, meanXp, meanXn):
    SizeMult = xp.shape[1]*xn.shape[1]
    SumSizeSquared = (xp.shape[1]+xn.shape[1])**2
    meansDifference = ((meanXp - meanXn))
    return (SizeMult/SumSizeSquared) * np.outer(meansDifference, meansDifference)

6. Compute between class scattering matrix Sb

In [9]:
Sb = betweenScatter(xp, xn, meanXp, meanXn)
print (Sb)

[[ 4.58224852e+00  4.66035503e+00 -5.20710059e-02]
 [ 4.66035503e+00  4.73979290e+00 -5.29585799e-02]
 [-5.20710059e-02 -5.29585799e-02  5.91715976e-04]]


7. Function to compute within class scattering matrix Sw

In [10]:
def withinScatter(xp, xn, covarXp, covarXn):
    xpLen = xp.shape[1]
    xnLen = xn.shape[1]
    totalLen = xpLen + xnLen
    return ((xpLen/totalLen) * covarXp) + ((xnLen/totalLen) * covarXn)

8. Compute within class scattering matrix Sw

In [11]:
Sw = withinScatter(xp, xn, covarXp, covarXn)
print (Sw)

[[ 3.52197802 -0.5467033  -0.57417582]
 [-0.5467033   2.75274725  0.02472527]
 [-0.57417582  0.02472527  0.57692308]]


9. Function to compute the LDA projection by solving the generalized eigenvalue decomposition problem

In [28]:
def LDAproject(Sw, Sb):
    eigenvalues, eigenvectors = np.linalg.eig(np.linalg.inv(Sw).dot(Sb))
    idx = np.argsort(eigenvalues)[::-1]
    eigenvectors = eigenvectors[idx]
    eigenvalues = eigenvalues[idx]
    highestEigenvector = eigenvectors[0]
    return highestEigenvector

10. Compute the LDA projection by solving the generalized eigenvalue decomposition problem

In [29]:
proj = LDAproject(Sw, Sb)
print(proj) 

[ 0.5789803  -0.68042478  0.01563448]


11. Function to perform binary LDA

In [30]:
def mybLDA_train(xp, xn):
    meanXp = computeMean(xp.T)
    meanXn = computeMean(xn.T)
    covarXp = covar(xp, meanXp)
    covarXn = covar(xn, meanXn)
    Sb = betweenScatter(xp, xn, meanXp, meanXn)
    Sw = withinScatter(xp, xn, covarXp, covarXn)
    proj = LDAproject(Sw, Sb)
    return proj

12. Function which takes a data matrix X and a projection direction v, returns a row vector r that has size as the number of rows in X, and r_i =+1 if the ith column of X is from the class as in Xp, and r_i =-1 if the ith column in X is from the class as in Xn

In [46]:
def mybLDA_classify(x,v):
    result = []
    for row in x.T:
        res = np.dot(row,v)
        if (res>0):
            result.append(1)
        else:
            result.append(-1)
    return np.array(result)

13. Run your function, mybLDA_train, on the data given in the problem setting, and then use the obtained projection direction and your function mybLDA_classify to classify the following data set

In [47]:
train = mybLDA_train(xp,xn)
x = np.array([[1.3,2.4,6.7,2.2,3.4,3.2], [8.1,7.6,2.1,1.1,0.5,7.4], [-1,2,3,-2,0,2]])
classify = mybLDA_classify(x,train)
print(classify)

[-1 -1  1  1  1 -1]
